In [ ]:
import os
import sys
import random
import warnings
import random as rn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize,rotate
from skimage.morphology import label
from skimage.color import rgb2gray

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import backend as K
from keras.losses import *

from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# Set some parameters
n_pixels = 128
channels = 1
path_train = '../input/train/'
path_test = '../input/test/'

In [ ]:
train=pd.read_csv("../input/train.csv")
print(train.shape)
train.head()

In [ ]:
sample=pd.read_csv("../input/sample_submission.csv")
print(sample.shape)
sample.head()

In [ ]:
depth=pd.read_csv("../input/depths.csv")
print(depth.shape)
depth.head()

In [ ]:
train_ids = train.id.values
test_ids = sample.id.values

In [ ]:
import skimage as sk

sk.__version__

In [ ]:
resize_mode='constant'

# Get and resize train images and masks
X = np.zeros((len(train_ids), n_pixels, n_pixels, channels), dtype=np.uint8)
y = np.zeros((len(train_ids), n_pixels, n_pixels, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    x = np.array(load_img(path + '/images/' + id_+".png",color_mode='grayscale'))
    x = resize(x, (n_pixels,n_pixels, channels), mode=resize_mode, 
               preserve_range=True,clip=True)
    X[n] = x
    mask = np.array(load_img(path + '/masks/' + id_+".png",color_mode='grayscale'))
    y[n] = resize(mask, (n_pixels,n_pixels,1), mode=resize_mode, 
                  preserve_range=True,clip=True)

print('Done!')

## TEST DATA

In [ ]:
# Get and resize test images
X_test = np.zeros((len(test_ids),n_pixels,n_pixels,channels), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    x= np.array(load_img(path + '/images/' + id_+".png",color_mode='grayscale'))
    x = resize(x, (n_pixels,n_pixels, 1), mode=resize_mode, 
               preserve_range=True,clip=True)
    X_test[n] = x

print('Done!')

## VALIDATION SPLIT AND STRATIFICATION

Calculating the salt coverage class

In [ ]:
coverage=np.zeros((train_ids.shape[0],))
for i,name in tqdm_notebook(enumerate(train_ids),total=train_ids.shape[0]):
    coverage[i]=np.sum(y[i,:,:,0])

train['coverage']=pd.Series(coverage)/y.shape[1]**2

print(train.shape)
train.head()

In [ ]:
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train["coverage_class"] = train.coverage.apply(cov_to_class)

print(train.shape)
train.head()

In [ ]:
## coverage class numpy array
coverage=train.coverage_class.values
print(coverage.shape)

In [ ]:
train.loc[train['coverage']==0].shape

In [ ]:
p=np.ones((4000,))
for i in range(4000):
    p[i]=np.sum(X[i,:,:,:])/(128*128)

index=[]
for j,i in enumerate(p):
    if i==0:
        index.append(j)
        
        
print(len(index))
print(index)

In [ ]:
q=np.ones((18000,))
for i in range(18000):
    q[i]=np.sum(X_test[i,:,:,:])/(128*128)
    
indextest=[]
for j,i in enumerate(q):
    if i==0:
        indextest.append(j)
        
print(len(indextest))
print(indextest)

## AUGMENTATION

In [ ]:
ind=3

p=rotate(X[ind,:,:,0].astype(np.uint8),angle=10,order=1,mode='reflect',
                clip=True,preserve_range=True)
q=rotate(y[ind,:,:,0].astype(np.uint8),angle=10,order=1,mode='reflect',
         clip=True,preserve_range=True)

plt.figure(figsize=(12,7))

plt.subplot(221)
plt.imshow(X[ind].reshape(128,128),cmap='gist_gray')
plt.subplot(222)
plt.imshow(y[ind].reshape(128,128),cmap='gist_gray')
plt.subplot(223)
plt.imshow(p.astype(np.uint8).reshape(128,128),cmap='gist_gray')
plt.subplot(224)
plt.imshow(q.astype(np.uint8).reshape(128,128),cmap='gist_gray')

plt.tight_layout()
plt.show()


In [ ]:
def crop_rescale_image(image,mask,state,limit):
    np.random.seed(state)
    h,w=image.shape[0],image.shape[1]
    dy=int(h*limit)
    y0=np.random.randint(int(0.1*h),dy)
    y1=h-y0
    
    
    np.random.seed(state+5000)
    dx=int(w*limit)
    x0=np.random.randint(int(0.1*w),dx)
    x1=w-x0
    
    m='reflect'
#     print(y0,x0)
    cropped_image=resize(image[y0:y1,x0:x1,:],(h,w),mode=m,
                         preserve_range=True)
    cropped_mask=resize(mask[y0:y1,x0:x1,:],(h,w),mode=m,
                        preserve_range=True)

    
    return cropped_image,cropped_mask
    
def random_crop_rescale(X,y,limit):
    m=X.shape[0]
    processed_image=np.zeros(X.shape)
    processed_mask=np.zeros(y.shape)
    for i in range(m):
        processed_image[i,:,:,:],processed_mask[i,:,:,:]=crop_rescale_image(X[i,:,:,:],y[i,:,:,:],i,limit)
    
    return processed_image.astype(np.uint8),processed_mask.astype(np.uint8)


def rotate_grp(images,masks,ang):
    rotated_images=np.zeros((images.shape))
    rotated_masks=np.zeros((masks.shape))
    for i in range(images.shape[0]):
        rotated_images[i,:,:,0]=rotate(X[i,:,:,0].astype(np.uint8),angle=ang,order=1,mode='reflect',
                clip=True,preserve_range=True)
        rotated_masks[i,:,:,0]=q=rotate(y[ind,:,:,0].astype(np.uint8),angle=ang,order=1,mode='reflect',
         clip=True,preserve_range=True)
        
        
    return rotated_images.astype(np.uint8),rotated_masks.astype(np.uint8)

In [ ]:
def augment(X,y,l,start_crop,end_crop,start_rot,end_rot,start_roll=-1,end_roll=-1,rolling=False):
    m=X.shape[0]
    print("Flipping")
    X=np.append(X,[np.fliplr(x) for x in X],axis=0)
    y=np.append(y,[np.fliplr(i) for i in y],axis=0)

    # trying random crop and rescale
    np.random.seed(42)
    index=np.arange(0,m)
    # shuffling the array
    np.random.shuffle(index)

    print("Random crop and resize")
    # random crop and resize
    processed_X,processed_y=random_crop_rescale(X[index[start_crop:end_crop]],
                                                y[index[start_crop:end_crop]],limit=l)
    X=np.append(X,processed_X,axis=0)
    y=np.append(y,processed_y,axis=0)

    print("Rotations")
    # 10 degree rotation
    rotated_X,rotated_y=rotate_grp(X[index[start_rot:end_rot]],
                                   y[index[start_rot:end_rot]],ang=10)

    X=np.append(X,rotated_X,axis=0)
    y=np.append(y,rotated_y,axis=0)
    
    if rolling:
        print("Rolling")
        
        roll_index=np.arange(0,X.shape[0])
        
        # shuffling
        np.random.shuffle(roll_index)
        
        # rolling train and mask
        to_roll_X=X[roll_index[start_roll:end_roll]]
        to_roll_y=y[roll_index[start_roll:end_roll]]
            
        # 40 steps rolling
        X=np.append(X,[np.roll(x, 40, axis = 1) for x in to_roll_X], axis = 0)
        y=np.append(y,[np.roll(t,40,axis=1) for t in to_roll_y],axis=0)
        
    return X,y

In [ ]:
# split

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1,stratify=coverage,random_state=42)

print(X_train.shape,X_val.shape)
print(y_train.shape,y_val.shape)

In [ ]:
print("Training Data Augmentation:")
print("Before Augmentation:",X_train.shape,y_train.shape)

X_train,y_train=augment(X_train,y_train,l=0.3,start_crop=0,end_crop=1000,start_rot=1000,
                        end_rot=2000,start_roll=0,end_roll=2000,rolling=True)

print("After Augmentation ",X_train.shape,y_train.shape)

print("Validation Augmentation:")
print("Before Augmentation:",X_val.shape,y_val.shape)

X_val,y_val=augment(X_val,y_val,l=0.3,start_crop=0,end_crop=150,start_rot=150,
                    end_rot=300,start_roll=0,end_roll=200,rolling=True)

print("After Augmentation:",X_val.shape,y_val.shape)

## Train Model
Metrics copied from public kernels

In [ ]:
## metric,copied from https://www.kaggle.com/jcesquiveld/tgs-simple-u-net-baseline

def castF(x):
    return K.cast(x, K.floatx())

def castB(x):
    return K.cast(x, bool)

def iou_loss_core(true,pred):
    intersection = true * pred
    notTrue = 1 - true
    union = true + (notTrue * pred)

    return (K.sum(intersection, axis=-1) + K.epsilon()) / (K.sum(union, axis=-1) + K.epsilon())

def competition_metric(true, pred): #any shape can go

    tresholds = [0.5 + (i*.05)  for i in range(10)]

    #flattened images (batch, pixels)
    true = K.batch_flatten(true)
    pred = K.batch_flatten(pred)
    pred = castF(K.greater(pred, 0.5))

    #total white pixels - (batch,)
    trueSum = K.sum(true, axis=-1)
    predSum = K.sum(pred, axis=-1)

    #has mask or not per image - (batch,)
    true1 = castF(K.greater(trueSum, 1))    
    pred1 = castF(K.greater(predSum, 1))

    #to get images that have mask in both true and pred
    truePositiveMask = castB(true1 * pred1)

    #separating only the possible true positives to check iou
    testTrue = tf.boolean_mask(true, truePositiveMask)
    testPred = tf.boolean_mask(pred, truePositiveMask)

    #getting iou and threshold comparisons
    iou = iou_loss_core(testTrue,testPred) 
    truePositives = [castF(K.greater(iou, tres)) for tres in tresholds]

    #mean of thressholds for true positives and total sum
    truePositives = K.mean(K.stack(truePositives, axis=-1), axis=-1)
    truePositives = K.sum(truePositives)

    #to get images that don't have mask in both true and pred
    trueNegatives = (1-true1) * (1 - pred1) # = 1 -true1 - pred1 + true1*pred1
    trueNegatives = K.sum(trueNegatives) 

    return (truePositives + trueNegatives) / castF(K.shape(true)[0])

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# loss
# here i will add dice and bce loss
# taken from https://www.kaggle.com/kmader/u-net-with-dice-and-augmentation

def dice_loss(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    score=K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
    return 1.0-score

def dice_and_bce(in_gt, in_pred):
    """combine DICE and BCE"""
    return binary_crossentropy(in_gt, in_pred) + dice_loss(in_gt, in_pred)

## ARCHITECUTE

In [ ]:
def batch_activate(x,normalize=True,do_activation=True):
    output=x
    if normalize:
        output=BatchNormalization()(output)
    if do_activation:
        output=Activation(activation='relu')(output)
        
    return output

def residual_block(inputs,fil,match_channel=False):
    x=batch_activate(inputs,normalize=True,do_activation=True)
    
    # 3x3 convolutions
    x=Conv2D(fil,(3,3),padding='same',activation=None,use_bias=False)(x)
    
    # batch norm and relu
    x=batch_activate(x,normalize=True,do_activation=True)
    
    # 3x3 convolutions
    x=Conv2D(fil,(3,3),padding='same',activation=None,use_bias=False)(x)
    
    if match_channel:
        inputs=Conv2D(fil,(1,1),padding='same',activation=None,use_bias=False)(inputs)
    
    p=Add()([x,inputs])
    
    return x

In [ ]:
inputs = Input(shape = (n_pixels,n_pixels, channels))

normalize=True
batch_normalize=True

if normalize:
    s=Lambda(lambda x: x / 255) (inputs)
else:
    s=inputs
    
ch=16
# c1=Conv2D(ch,(3,3),activation=None,use_bias=False,padding='same')(s)

# 128 to 64
c1 = residual_block(s,ch,match_channel=False)
c1 = residual_block(c1,ch,match_channel=False)
c1=  BatchNormalization()(c1) if batch_normalize else c1
p1 = MaxPooling2D(pool_size=(2, 2))(c1)

# 64 to 32
c2 = residual_block(p1,2*ch,match_channel=True)
c2 = residual_block(c2,2*ch,match_channel=False)
c2 = BatchNormalization()(c2) if batch_normalize else c2
p2 = MaxPooling2D(pool_size = (2, 2))(c2)

# 32 to 16
c3 = residual_block(p2,4*ch,match_channel=True)
c3 = residual_block(c3,4*ch,match_channel=False)
c3= BatchNormalization()(c3) if batch_normalize else c3
p3 = MaxPooling2D(pool_size = (2, 2))(c3)


# 16 to 8
c4 = residual_block(p3,8*ch,match_channel=True)
c4 = residual_block(c4,8*ch,match_channel=False)
c4=BatchNormalization()(c4) if batch_normalize else c4
p4 = MaxPooling2D(pool_size=(2, 2))(c4)


# 16 to 8
c5 = residual_block(p4, 16*ch, match_channel=True)
c5 = residual_block(c5, 16*ch, match_channel=False)
c5=BatchNormalization()(c5) if batch_normalize else c5

# 8 to 16
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), activation = "relu",padding = "same")(c5)
u6 = concatenate([u6, c4]) # 256 shape
c6 = residual_block(u6,8*ch,match_channel=True)
c6 = residual_block(c6,8*ch,match_channel=False)
c6 = BatchNormalization()(c6) if batch_normalize else c6

# 16 to 32
u7 = Conv2DTranspose(64, (2, 2), strides = (2, 2,), activation = "relu",padding = "same")(c6)
u7 = concatenate([u7, c3]) # 128  shape
c7 = residual_block(u7,4*ch,match_channel=True)
c7 = residual_block(c7,4*ch,match_channel=False)
c7 = BatchNormalization()(c7) if batch_normalize else c7

# 32 to 64
u8 = Conv2DTranspose(32, (2, 2), strides = (2, 2), activation = "relu",padding = "same")(c7)
u8 = concatenate([u8, c2]) # 64 shape
c8 = residual_block(u8,2*ch,match_channel=True)
c8 = residual_block(c8,2*ch,match_channel=False)
c8 = BatchNormalization()(c8) if batch_normalize else c8

# 64 to 128
u9 = Conv2DTranspose(16, (2, 2), strides = (2, 2), activation = "relu",padding = "same")(c8)
u9 = concatenate([u9, c1])   # 32 shape
c9 = residual_block(u9,ch,match_channel=True)
c9 = residual_block(c9,ch,match_channel=False)
c9 = BatchNormalization()(c9) if batch_normalize else c9

outputs = Conv2D(1, (1, 1), activation = "sigmoid")(c9)

model = Model(inputs = [inputs], outputs = [outputs])
optimizer = Adam()

model.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = [competition_metric])
model.summary()


In [ ]:

## copied from https://github.com/jfpuget/LibFM_in_Keras/blob/master/keras_blog.ipynb
# if we do this we cant use mean_iou as the metric due to initializations errors
# try:
#     del sess
# except:
#     pass
# sess = init_seeds(0)


batch=32
ep=50
name_model='resnet34_unet.h5'
include_earlystopper=True
include_checkpointer=True
include_lr=True



earlystopper = EarlyStopping(monitor='val_competition_metric',patience=10,
                             mode='max',verbose=1)
checkpointer = ModelCheckpoint(name_model, monitor='val_competition_metric',mode='max',
                               verbose=1, save_best_only=True)
reduce_lr=ReduceLROnPlateau(monitor='val_competition_metric',patience=5, 
                            min_lr=0.00001,mode='max',verbose=1,factor=0.5)


callback=[]

if include_earlystopper:
    callback.append(earlystopper)

if include_checkpointer:
    callback.append(checkpointer)
    
if include_lr:
    callback.append(reduce_lr)



results = model.fit(X_train, y_train, validation_data=[X_val,y_val], batch_size=batch, epochs=ep, 
                    callbacks=callback)

In [ ]:
# batch=16
# ep=15


# model = load_model(name_model, custom_objects={'competition_metric': competition_metric,
#                                             'mean_iou':mean_iou,'dice_and_bce':dice_and_bce})

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', 
#                   metrics=[mean_iou,competition_metric])

# results = model.fit(X_train, y_train, validation_data=[X_val,y_val], 
#                     batch_size=batch, epochs=ep, callbacks=callback)

# Test Data
First we'll get the test data. This takes a while, it's 18000 samples.

## EVALUATION

In [ ]:
model = load_model(name_model, custom_objects={'competition_metric': competition_metric,
                                               'mean_iou':mean_iou,'dice_and_bce':dice_and_bce})

print(model.metrics_names)
print(model.evaluate(X_train,y_train))
print(model.evaluate(X_val,y_val))

In [ ]:
# Predict on train, val and test
preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_val, verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
thre=0.5
preds_train_t = (preds_train > thre).astype(np.uint8)
preds_val_t = (preds_val > thre).astype(np.uint8)
preds_test_t = (preds_test > thre).astype(np.uint8)

In [ ]:
# amit's metric 

# https://www.kaggle.com/amitkvikram

thresholds = [0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
n_thresholds = len(thresholds)

#function to return A∩B and A∪B
def IoUhelper(TrueMask, predictedMask):
    intersection = cv2.bitwise_and(TrueMask, predictedMask)
    union = cv2.bitwise_or(TrueMask, predictedMask)
    intersectionCnt = cv2.countNonZero(intersection)
    unionCnt = cv2.countNonZero(union)
    return intersectionCnt, unionCnt

def meanHit(TrueMask, predictedMask):
    hitCnt = 0
    intersectionCnt, unionCnt = IoUhelper(TrueMask, predictedMask)
#     print("intersction = ", intersectionCnt, "unionCnt = ", unionCnt)
    #if both TrueMask is empty and PredictedMask is empty
    if(intersectionCnt == 0 and unionCnt == 0):
        return 1
    
    #if TrueMask in empty and  Predicted mask in non empty
    #--------------------OR-------------------------
    #if TrueMask is non empty and Predicted Mask is empty
    if(intersectionCnt == 0 and unionCnt != 0):
        return 0
    
    #if TrueMask is non empty and predicted Mask is non emtpy
    IoU = intersectionCnt/unionCnt
    for t in thresholds:
        hitCnt+=int(IoU > t)
    return hitCnt/n_thresholds


tot=0
for i in range(X_val.shape[0]):
    tot=tot+meanHit(y_val[i].astype(np.uint8),preds_val_t[i].astype(np.uint8))
    

print("The validatoin IOU is ",tot/X_val.shape[0])

In [ ]:
# Create list of upsampled test masks
preds_test_upsampled = []
for i in tnrange(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (101, 101), mode='constant',
                                       preserve_range=True))
    

# Prepare Submission
We need to prepare the submission. 

In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {fn:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')

In [ ]:
sub.head()

In [ ]:
sub.loc[sub['rle_mask']==""].shape